<a href="https://colab.research.google.com/github/Superserx/Superserx/blob/main/sd_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **StableDiffusion GColab version without any restrictions**

GitHub.png **[GitHub](https://github.com/AlexSgt/lite_colab)**

In [ ]:
#@markdown # Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

In [ ]:
#@title 1. Run this cell to install requirments

from IPython.display import clear_output

print('[1;32mInstalling requirements...')
#%pip install --quiet blendmodes accelerate basicsr fonts font-roboto gfpgan gradio invisible-watermark numpy omegaconf opencv-contrib-python requests piexif Pillow pytorch-lightning==1.7.7 realesrgan scikit-image==0.19 timm==0.4.12 transformers==4.25.1 torch einops jsonmerge clean-fid resize-right torchdiffeq kornia lark inflection GitPython torchsde safetensors psutil rich
#%cd /content/
!git clone -q --branch main https://github.com/Superserx/sd_colab
%cd /content/sd_colab
%pip install -r requirements.txt --quiet
!sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" /content/lite_colab/modules/sd_models.py
!sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' /content/lite_colab/startfk.py
!sed -i "s@map_location='cpu'@map_location='cuda'@" /content/sd_colab/modules/extras.py
!cp /content/sd_colab/assets/blocks.py /usr/local/lib/python3.10/dist-packages/gradio/blocks.py

clear_output()
print('[1;32mDone!')
#!python launch.py --share --theme dark --api --ckpt-dir /content/models/ --disable-safe-unpickle --enable-insecure-extension-access --opt-sdp-attention --disable-console-progressbars --no-download-sd-model

In [ ]:
#@title 2. Download Model
Model_to_download="Anythingv3" #@param [ "Anythingv3", "WaifuDiffusion-Vae", "Eimis", "EvtV3", "ACertain", "ElysiumV3", "Wlop-any", "LeafMix", "Abyss7thLayer", "Healys", "LigneClaire", "KriboMix", "InizioSkinjob", "PantyStocking", "WaifuBodyBlenderMix", "StuffyMix", "AbyssOrangeMix2", "AbyssOrangeMix3", "8528", "AmbrosiaFusion", "AniMeth", "AnythingV4.0", "ArchonInstagram", "CounterveitV2.5", "CounterveitV2.2", "CounterveitV2.2", "HD-17", "HDDreamGape", "MomochaMix", "Mozarella", "PadoruDiffusion", "PastelMix", "Protothing", "Shirayuki", "NutMegMix", "KaniAnime", "EverythingMix", "BlossomExtract", "HentaiElysium", "BerryMix", "Systemy-CSR", "ElyOrangeMix", "AnyTwamMix", "DarkberryMix", "Grapefruit", "Mitsudoue"]
#@markdown Confused on what model to choose? Look for examples and comparison [here](https://github.com/etherealxx/etherportal-webui-colab/blob/master/ModelComparison.md).
#@markdown <br/>If the download speed is slow (Below 10MiB/s), press the stop button then re-run the cell)
#@markdown <br/><br/>If the model you wanted isn't listed, and you have the direct link, paste it to the form below. (Supported link: Huggingface, MEGA)
optional_direct_link='' #@param {type:"string"}

import os
import urllib.request, subprocess, contextlib #these handle mega.nz
from IPython.display import clear_output #this also handle mega.nz
from IPython.utils import capture
import requests
from tqdm import tqdm

folder = '/content/stable-diffusion-webui/models/Stable-diffusion/'

def checkdownloaded(pathx):
  if os.path.exists(pathx) or (os.path.exists(os.path.join(folder, 'Anything-V3.0.safetensors')) and Model_to_download=="Anythingv3"):

    if pathx.endswith("tmp"):
      os.remove(pathx)
    else:
      print(f'[1;32mThat model is already downloaded.')
      print('[0m')
      return 2

#civitstuff
checkerpath = os.path.join(folder, 'civitmodel.txt')
alreadythere = False

def checkearlycivit():
    alreadythere = False
    lines = []
    if not os.path.exists(checkerpath):
        !touch {checkerpath}
    #check for unavailable model first
    if os.stat(checkerpath).st_size != 0:
        with open(checkerpath, "r") as file:
            for line in file:
                key, value = line.strip().split(" = ")
                modelfile = os.path.join(folder, value)
                if os.path.exists(modelfile):
                    lines.append(line)
                    if key == Model_to_download:
                        alreadythere = True
                else:
                    print(f"Seems like {key} model is not found on the model folder.")
                    print()

        with open(checkerpath, "w") as file:
            file.writelines(lines)

    if alreadythere == True: return 2

def civitgetpath(modelname):
    with open(checkerpath, "r") as file:
        for line in file:
            key, value = line.strip().split(" = ")
            if key == modelname:
                return os.path.join(folder, value)

def civitwrite(modelname):
    towrite = f"{Model_to_download} = {modelname}"
    with open(checkerpath, "a") as file:
        file.write(towrite + "\n")
#civitstuffend

def download_hf(link):
  filename = link.rsplit('/', 1)[-1]
  if Model_to_download == 'WaifuDiffusion-Vae':
    filename = 'WD-vae.vae.pt'
  elif Model_to_download == 'Anythingv3':
    filename = 'Anything-V3.0.safetensors'

  hfpath = os.path.join(folder, filename)
  check = checkdownloaded(hfpath)
  if check == 2:
    if Model_to_download=='Anythingv3' and not os.path.exists(os.path.join(folder, 'Anything-V3.0.vae.pt')):
      !gdown https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -O /content/stable-diffusion-webui/models/Stable-diffusion/Anything-V3.0.vae.pt
    return

  print(f'\033[1;32mDownloading {Model_to_download} Model...\033[0m')
  !gdown {link} -O /content/stable-diffusion-webui/models/Stable-diffusion/{filename}
  if Model_to_download=='Anythingv3':
    !gdown https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -O /content/stable-diffusion-webui/models/Stable-diffusion/Anything-V3.0.vae.pt
  print(f'\033[1;32mModel Downloaded.\033[0m')

def download_civit(url):
    print(f'\033[1;32mDownloading {Model_to_download} Model...\033[0m')

    head_response = requests.head(url)
    total = int(head_response.headers.get('Content-Length', 0))
    civitcheck = checkearlycivit()
    if civitcheck == 2: #model exist
        civitpath = civitgetpath(Model_to_download)
        downloaded_size = os.path.getsize(civitpath)
        if total != 0 and downloaded_size != total:
            os.remove(civitpath)
            checkearlycivit()
            print(f'Removed incomplete download: {Model_to_download}')
        else:
            print(f'[1;32mThat model is already downloaded.')
            print('[0m')
            return

    downloaded_size = 0

    if not civitcheck == 2:
        pathtemp = os.path.join(folder, Model_to_download + ".bdgh")
        print(f'The model "{Model_to_download}" will be renamed correctly after downloading')
        with open(pathtemp, 'wb') as file, tqdm(
            desc=pathtemp,
            total=total,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
            initial=downloaded_size,
        ) as bar:
            resp = requests.get(url, stream=True)

            for data in resp.iter_content(chunk_size=1024):
                size = file.write(data)
                bar.update(size)
        # Check if the file has been downloaded successfully
        downloaded_size = os.path.getsize(pathtemp)
        if (total != 0 and downloaded_size != total) or (downloaded_size < 1000):
            # Remove the file if it has not been downloaded successfully
            os.remove(pathtemp)
            print()
            if downloaded_size < 1000:
                print(f'The model "{Model_to_download}" is probably had the link changed. Contact the owner of this repo.')
            else:
                print(f'Removed incomplete download: {Model_to_download}')
            return
        actualfilename = resp.headers['Content-Disposition'].split("filename=")[1].strip('"')
        #%cd {folder}
        os.rename(pathtemp, os.path.join(folder, actualfilename))
        print (f'Model "{Model_to_download}.bdgh" renamed to "{actualfilename}"')
        civitwrite(actualfilename)
        print(f'\033[1;32mModel Downloaded.\033[0m')


print('[1;32mInstalling webui...')
with capture.capture_output() as cap:
  %cd /content
  !if ! [ -d stable-diffusion-webui ]; then git clone -b v1.6 https://github.com/camenduru/stable-diffusion-webui; touch forkedwebui; fi
print('[0m')
!if [ -f {folder}'Put Stable Diffusion checkpoints here.txt' ]; then rm {folder}'Put Stable Diffusion checkpoints here.txt'; fi

%cd {folder}
    #sfw
if Model_to_download=='Anythingv3':
  download_hf('https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp32-pruned.safetensors')
if Model_to_download=='WaifuDiffusion-Vae':
  download_hf('https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt')
if Model_to_download=='Eimis':
  download_hf('https://huggingface.co/eimiss/EimisAnimeDiffusion_1.0v/resolve/main/EimisAnimeDiffusion_1-0v.ckpt')
if Model_to_download=='EvtV3':
  download_hf('https://huggingface.co/haor/Evt_V3/resolve/main/Evt_V3_ema.ckpt')
if Model_to_download=='ACertain':
  download_hf('https://huggingface.co/JosephusCheung/ACertainModel/resolve/main/ACertainModel.ckpt')
if Model_to_download=='ElysiumV3':
  download_hf('https://huggingface.co/hesw23168/SD-Elysium-Model/resolve/main/Elysium_Anime_V3.safetensors')
if Model_to_download=='Wlop-any':
  download_hf('https://huggingface.co/SirVeggie/wlop/resolve/main/wlop-any.ckpt')
if Model_to_download=='LeafMix':
  download_hf('https://huggingface.co/etherealxx/anyleaf-temp/resolve/main/Leaf-pruned.safetensors')
if Model_to_download=='Abyss7thLayer':
  download_hf('https://huggingface.co/syaimu/7th_Layer/resolve/main/7th_layer/Abyss_7th_layer.ckpt')
if Model_to_download=='Healys':
  download_civit('https://civitai.com/api/download/models/1498')
if Model_to_download=='LigneClaire':
  download_hf('https://huggingface.co/breakcore2/ligne_claire_anime_diffusion/resolve/main/ligne_claire_anime_diffusion_v1.safetensors')
if Model_to_download=='KriboMix':
  download_civit('https://civitai.com/api/download/models/1289')
if Model_to_download=='InizioSkinjob':
  download_hf('https://huggingface.co/Cinnamomo/inizio/resolve/main/safetensor/inizio_skinjob.safetensors')
if Model_to_download=='PantyStocking':
  download_civit('https://civitai.com/api/download/models/1560')
if Model_to_download=='WaifuBodyBlenderMix':
  download_civit('https://civitai.com/api/download/models/1838')
if Model_to_download=='StuffyMix':
  download_civit('https://civitai.com/api/download/models/3376')
if Model_to_download=='AbyssOrangeMix2':
  download_hf('https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors')
if Model_to_download=='AbyssOrangeMix3':
  download_hf('https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3\(orangemixs\).safetensors')
if Model_to_download=='8528':
  download_hf('https://huggingface.co/ckpt/8528-diffusion/resolve/main/8528d-final.ckpt')
if Model_to_download=='AmbrosiaFusion':
  download_hf('https://huggingface.co/NiteStormz/ProtoThing_200/resolve/main/AmbrosiaFusion.ckpt')
if Model_to_download=='AniMeth':
  download_hf('https://huggingface.co/Rasgeath/self_made_sauce/resolve/main/AniMeth.safetensors')
if Model_to_download=='AnythingV4.0':
  download_hf('https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0-pruned.safetensors')
if Model_to_download=='ArchonInstagram':
  download_hf('https://huggingface.co/misobarisic/playground/resolve/main/ArchonInstagram-fp16.ckpt')
if Model_to_download=='CounterveitV2.5':
  download_hf('https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5_pruned.safetensors')
if Model_to_download=='CounterveitV2.2':
  download_hf('https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.2.safetensors')
if Model_to_download=='CounterveitV2.2':
  download_hf('https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.2.safetensors')
if Model_to_download=='HD-17':
  download_hf('https://huggingface.co/Deltaadams/HentaiDiffusion/resolve/main/HD-17.ckpt')
if Model_to_download=='HDDreamGape':
  download_hf('https://huggingface.co/Horsebird/HDDream-G/resolve/main/HDDream-Gape.safetensors')
if Model_to_download=='MomochaMix':
  download_hf('https://huggingface.co/NoCrypt/momocha-mix/resolve/main/momocha-6E7-0.315-AD-E7-0.15-WS.ckpt')
if Model_to_download=='Mozarella':
  download_hf('https://huggingface.co/Rasgeath/self_made_sauce/resolve/main/Mozzarella-pruned.safetensors')
if Model_to_download=='PadoruDiffusion':
  download_hf('https://huggingface.co/misobarisic/playground/resolve/main/padoruDiffusion_1.ckpt')
if Model_to_download=='PastelMix':
  download_hf('https://huggingface.co/andite/pastel-mix/resolve/main/pastelmix-fp32.safetensors')
if Model_to_download=='Protothing':
  download_hf('https://huggingface.co/NiteStormz/ProtoThing_200/resolve/main/ProtoThing_200.ckpt')
if Model_to_download=='Shirayuki':
  download_hf('https://huggingface.co/hesw23168/SD_Shirayuki_Model/resolve/main/Shirayuki_Anime_V1-fp16.safetensors')
if Model_to_download=='NutMegMix':
  download_hf('https://huggingface.co/acheong08/nutmegmix/resolve/main/nutmegmix.safetensors')
if Model_to_download=='KaniAnime':
  download_hf('https://huggingface.co/Rasgeath/self_made_sauce/resolve/main/Kani-anime-pruned.ckpt')
if Model_to_download=='EverythingMix':
  download_hf('https://huggingface.co/DrBob2142/Everything/resolve/main/Everything%20V1B.safetensors')
if Model_to_download=='BlossomExtract':
  download_hf('https://huggingface.co/NUROISEA/anything-mix/resolve/main/blossom-extract.safetensors')
if Model_to_download=='HentaiElysium':
  download_hf('https://huggingface.co/NUROISEA/anything-mix/resolve/main/hentai-elysium-50.safetensors')

    #nsfw
if Model_to_download=='BerryMix':
  download_hf('https://huggingface.co/misobarisic/berry-mix/resolve/main/berry_mix.ckpt')
if Model_to_download=='Systemy-CSR':
  download_hf('https://huggingface.co/etherealxx/systemy-csrmodel-cutesexyrobutts/resolve/main/CSRmodel_1-CSRmodel_0-Weighted_sum-merged.ckpt')
if Model_to_download=='ElyOrangeMix':
  download_hf('https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/ElyOrangeMix/ElyOrangeMix.ckpt')
if Model_to_download=='AnyTwamMix':
  download_civit('https://civitai.com/api/download/models/4527')
if Model_to_download=='DarkberryMix':
  download_civit('https://civitai.com/api/download/models/1992')
if Model_to_download=='Grapefruit':
  download_civit('https://civitai.com/api/download/models/6360')
if Model_to_download=='Mitsudoue':
  download_civit('https://civitai.com/api/download/models/1454')

#these code below handle mega.nz
def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", optional_direct_link, folder]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
    )
    for line in unbuffered(proc):
      if not line.startswith("Download"):
        print(f"\r{line}", end="")
      else:
        print(f"\n{line}")


def installmega():
  HOME = os.path.expanduser("~")
  if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
      hCode = "https://raw.githubusercontent.com/biplobsd/" \
                  "OneClickRun/master/res/ocr.py"
      urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

  from ocr import (
      runSh,
      loadingAn,
  )

  if not os.path.exists("/usr/bin/mega-cmd"):
      loadingAn()
      print("Installing MEGA ...")
      runSh('sudo apt-get -y update')
      runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
      runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
      runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
      print("MEGA is installed.")
      clear_output()
#these code above handle mega.nz

if optional_direct_link!='':
  if optional_direct_link.startswith("https://huggingface.co/"):
    custom_download = os.path.basename(optional_direct_link)
    custom_path = os.path.join(folder,custom_download)
    if not os.path.isfile(custom_path):
      print(f'[1;32mDownloading {custom_download}')
      print('[0m')
      !gdown {optional_direct_link} -O {custom_path}
    else:
      print(f'[1;32mThat custom model is already downloaded.')
      print('[0m')
  elif optional_direct_link.startswith("https://mega.nz/"):
    print(f'[1;32mDownloading from a Mega link')
    print('[0m')
    installmega()
    newlines = ['\n', '\r\n', '\r']
    transfare()

files = os.listdir(folder)
for file in files:
    if file.endswith('tmp') or file.endswith('bdgh'):
        file_path = os.path.join(folder, file)
        print(f"Removed incomplete download: {file_path}")
        os.remove(file_path)

for root, dirs, files in os.walk(folder):
    for file in files:
        if file.endswith('.ckpt') or file.endswith('.safetensors'):
            checksizepath = os.path.join(root, file)
            MB = 1000000 # 1 megabyte = 1000000 bytes
            lorapath = '/content/stable-diffusion-webui/models/Lora'
            if os.path.getsize(checksizepath) < 200 * MB:
                if os.path.getsize(checksizepath) < MB:
                    os.remove(checksizepath)
                    print()
                    print(f'The model {checksizepath} is probably had the link changed. Contact the owner of this repo.')
                else:
                    if not os.path.exists(lorapath):
                        os.makedirs(lorapath)
                    os.rename(checksizepath, os.path.join(lorapath, file))
                    print()
                    print(f'The model {checksizepath} is probably a Lora, so it is moved to lora folder')

print()
print('[1;32mList of installed models:')
print('[0m')
!ls *.{ckpt,safetensors,pt} {folder} 2>/dev/null

In [ ]:
#@markdown # Start SD

!python launch.py --share --api --disable-safe-unpickle --enable-insecure-extension-access --opt-sdp-attention --disable-console-progressbars --no-download-sd-model

In [ ]:
#@markdown # Save "Outputs" to GDrive

from google.colab import drive
drive.mount('/content/gdrive')

!mkdir -p /content/gdrive/MyDrive/sd_colab/outputs
!cp -r /content/sd_colab/outputs /content/gdrive/MyDrive/sd_colab

clear_output()
print('[1;32mDone!')